In [ ]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, Slider, CustomJS, LabelSet, CheckboxButtonGroup
from bokeh.layouts import column
import pandas as pd

asthma = pd.read_csv("NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
pm25 = pd.read_csv("NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")

asthma['Percent_clean'] = asthma['Percent'].str.extract(r'(\d+\.\d+)').astype(float)
asthma['TimePeriod'] = asthma['TimePeriod'].astype(int)

pm25['Annual_mean'] = pm25['Annual mean mcg/m3']
pm25['TimePeriod'] = pm25['TimePeriod'].astype(int)

common_years = set(asthma['TimePeriod']) & set(pm25['TimePeriod'])

asthma_yearly_filtered = asthma[asthma['TimePeriod'].isin(common_years)].reset_index(drop=True)
pm25_yearly_filtered = pm25[pm25['TimePeriod'].isin(common_years)].reset_index(drop=True)

asthma_yearly = asthma_yearly_filtered.groupby('TimePeriod')['Percent_clean'].mean().reset_index()
pm25_yearly = pm25_yearly_filtered.groupby('TimePeriod')['Annual_mean'].mean().reset_index()

asthma_yearly['Percent_norm'] = (asthma_yearly['Percent_clean'] - asthma_yearly['Percent_clean'].min()) / (asthma_yearly['Percent_clean'].max() - asthma_yearly['Percent_clean'].min())

pm25_yearly['Cigarettes_per_day'] = pm25_yearly['Annual_mean'] / 22
pm25_yearly['Cigarettes_per_year'] = (pm25_yearly['Cigarettes_per_day'] * 365).round(0).astype(int)

source_asthma = ColumnDataSource(asthma_yearly)
source_pm25 = ColumnDataSource(pm25_yearly)

source_asthma_shifted = ColumnDataSource(data=dict(TimePeriod=asthma_yearly['TimePeriod'], Percent_norm=asthma_yearly['Percent_norm']))

p = figure(title='Normalized Asthma vs PM2.5 (Cigarettes Equivalent + Labels)',
           x_axis_label='Year',
           y_axis_label='Normalized Asthma (0-1) / PM2.5 as Cigarettes/day',
           width=900, height=500,
           tools='pan,wheel_zoom,box_zoom,reset,save')

bars_pm25 = p.vbar(x='TimePeriod', top='Cigarettes_per_day', source=source_pm25,
                   width=0.8, color='#9467bd', legend_label='PM2.5 (Cigarettes/day)', alpha=0.6)

labels = LabelSet(x='TimePeriod', y='Cigarettes_per_day', text='Cigarettes_per_year',
                  level='glyph', x_offset=-13, y_offset=3, source=source_pm25,
                  text_font_size="10pt", text_color="#000000",
                  background_fill_color='white', background_fill_alpha=0.6,
                  visible=True)
p.add_layout(labels)

line1 = p.line('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', legend_label='Asthma (Normalized)', line_width=3)
circle1 = p.circle('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', size=8)

hover_asthma = HoverTool(tooltips=[('Year', '@TimePeriod'), ('Asthma Value (normalized)', '@Percent_norm{0.00}')], renderers=[line1, circle1])
hover_pm25 = HoverTool(tooltips=[('Year', '@TimePeriod'), ('PM2.5 (µg/m³)', '@Annual_mean{0.0}'), ('Cigarettes/day', '@Cigarettes_per_day{0.00}')], renderers=[bars_pm25])
p.add_tools(hover_asthma, hover_pm25)

p.legend.click_policy = "hide"

label_toggle = CheckboxButtonGroup(labels=["Show Cigarettes/year Labels"], active=[0])
callback_label = CustomJS(args=dict(labels=labels, button=label_toggle), code="""
    labels.visible = button.active.includes(0);
""")
label_toggle.js_on_change('active', callback_label)

shift_slider = Slider(start=-10, end=10, value=0, step=1, title="Shift Asthma Curve (Years)")
callback = CustomJS(args=dict(source=source_asthma_shifted, slider=shift_slider,
                               original_x=asthma_yearly['TimePeriod'].tolist(),
                               original_y=asthma_yearly['Percent_norm'].tolist()), code=""" 
    const data = source.data;
    const shift = slider.value;
    const x = original_x;
    const y = original_y;
    const new_x = [];
    for (let i = 0; i < x.length; i++) {
        new_x.push(x[i] + shift);
    }
    data['TimePeriod'] = new_x;
    source.change.emit();
""")
shift_slider.js_on_change('value', callback)

output_file("asthma_pm25_cigarettes_labels_final.html", title="Asthma vs PM2.5 (Cigarettes Equivalent + Labels)")

save(column(p, shift_slider, label_toggle))


'c:\\Users\\Eva María\\OneDrive - Danmarks Tekniske Universitet\\master\\1_year\\spring\\socialData\\finalprojectRepo\\ProjectB_Group55.github.io\\code and data\\asthma_pm25_cigarettes_labels_final.html'